In [105]:
import pandas as pd
import mplfinance

In [106]:
df = pd.read_csv("etps.csv", delimiter=";")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
for val in df.columns:
    df[val] = df[val].fillna(0)

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   FundLongName                71 non-null     object 
 1   ValorSymbol                 71 non-null     object 
 2   FundReutersTicker           71 non-null     object 
 3   FundBloombergTicker         71 non-null     object 
 4   ValorNumber                 71 non-null     int64  
 5   ISIN                        71 non-null     object 
 6   IssuerLongNameDesc          71 non-null     object 
 7   IssuerNameFull              71 non-null     object 
 8   TradingBaseCurrency         71 non-null     object 
 9   FundCurrency                71 non-null     object 
 10  ManagementFee               71 non-null     float64
 11  ReplicationMethodDesc       71 non-null     object 
 12  ManagementStyleDesc         71 non-null     object 
 13  MarketMakers                71 non-nu

In [108]:
df.head(3)

,FundLongName,ValorSymbol,FundReutersTicker,FundBloombergTicker,ValorNumber,ISIN,IssuerLongNameDesc,IssuerNameFull,TradingBaseCurrency,FundCurrency,ManagementFee,ReplicationMethodDesc,ManagementStyleDesc,MarketMakers,ClosingPrice,ClosingPerformance,ClosingDelta,FundUnderlyingDescription,BidVolume,BidPrice,AskVolume,AskPrice,MidSpread,PreviousClosingPrice,MarketDate,MarketTime,OpeningPrice,OpeningPrice.1,DailyLowPrice,OnMarketVolume,OffBookVolume,TotalTurnover,TotalTurnoverCHF,ProductLineDesc,AssetClassDesc,UnderlyingGeographicalDesc,LegalStructureCountryDesc,UnderlyingProviderDesc
0,21Shares Algorand (ALGO) ETP,ALGO,ALGO.S,ALGO SE,114688231,CH1146882316,21Shares AG,21Shares AG,CHF,USD,2.5,physical,passive,Flow Traders B.V.,1.8088,-10.79,-0.2187,0,0,0,0,0.0,0.0,2.0275,20221219,192200,0.0,0.0,0.0,0,0,0.0,0.0,Exchange Traded Product,Crypto,Global,Switzerland,0
1,21Shares Algorand (ALGO) ETP,ALGO,ALGO.S,ALGO SE,114688231,CH1146882316,21Shares AG,21Shares AG,EUR,USD,2.5,physical,passive,Flow Traders B.V.,1.8310,-10.64,-0.2180,0,0,0,0,0.0,0.0,2.0490,20221219,192200,0.0,0.0,0.0,0,0,0.0,0.0,Exchange Traded Product,Crypto,Global,Switzerland,0
2,21Shares Algorand (ALGO) ETP,ALGO,ALGO.S,ALGO SE,114688231,CH1146882316,21Shares AG,21Shares AG,USD,USD,2.5,physical,passive,Flow Traders B.V.,1.9420,-9.76,-0.2100,0,0,0,0,0.0,0.0,2.1520,20221219,192200,0.0,0.0,0.0,0,0,0.0,0.0,Exchange Traded Product,Crypto,Global,Switzerland,0


##### Target Fields 
- ValorSymbol
- MarketDate
- TradingBaseFiat
- OpeningPrice
- ClosingPrice
- ClosingPerformance
- ClosingDelta

In [109]:
shares21 = df[["ValorSymbol", "MarketDate","TradingBaseCurrency", "OpeningPrice", 
            "ClosingPrice", "ClosingPerformance","ClosingDelta"]].copy() 

shares21 = shares21.rename(columns={'ValorSymbol': 'Symbol', 'MarketDate': 'Date',
                                    "TradingBaseCurrency":"Fiat", "OpeningPrice":"Open",
                                    "ClosingPrice":"Close", "ClosingPerformance":"Performance",
                                    "ClosingDelta":"Delta" })

shares21['Date'] = pd.to_datetime(shares21['Date'], format='%Y%m%d')
shares21.head(10)

,Symbol,Date,Fiat,Open,Close,Performance,Delta
0,ALGO,2022-12-19,CHF,0.0000,1.8088,-10.79,-0.2187
1,ALGO,2022-12-19,EUR,0.0000,1.8310,-10.64,-0.2180
2,ALGO,2022-12-19,USD,0.0000,1.9420,-9.76,-0.2100
3,AVAX,2022-12-19,CHF,2.1785,2.1785,-4.99,-0.1145
4,AVAX,2022-12-19,EUR,0.0000,2.1905,-5.28,-0.1220
5,AVAX,2022-12-19,GBP,0.0000,1.9084,-5.29,-0.1066
6,AVAX,2022-12-19,USD,2.3440,2.3440,-6.24,-0.1560
7,ABNB,2022-12-19,USD,17.7480,17.7920,-0.70,-0.1260
8,ABCH,2022-12-19,USD,4.4945,4.4790,-2.13,-0.0975
9,CBTC,2022-12-19,USD,0.0000,3.9945,-1.46,-0.0590


In [110]:
shares21.describe()

,Open,Close,Performance,Delta
count,71.000000,71.000000,71.000000,71.000000
mean,2.524194,18.119689,-3.501408,-0.342772
std,4.551744,93.509266,3.285612,1.211855
min,0.000000,0.002900,-12.870000,-10.300000
25%,0.000000,4.259750,-5.635000,-0.260000
50%,0.000000,5.700000,-2.720000,-0.177500
75%,4.496500,9.754500,-1.525000,-0.097500
max,19.520000,793.900000,6.250000,0.316000


#### Key Metrics
- Avg by Fiat
- Avg by Symbol

In [111]:
shares21.groupby(['Fiat']).mean().sort_values(by="Open", ascending=False)

,Open,Close,Performance,Delta
Fiat,,,,
USD,4.194677,8.061230,-2.881667,-0.181630
CHF,2.450306,6.317039,-4.286111,-0.235567
GBP,0.433955,6.076664,-4.173636,-0.207927
EUR,0.408955,6.382818,-3.437273,-0.187318
JPY,0.000000,793.900000,-1.280000,-10.300000


In [113]:
shares21.groupby(['Symbol']).mean().sort_values(by="Open", ascending=False)

,Open,Close,Performance,Delta
Symbol,,,,
SHETH,19.520000,19.470000,1.650000,0.316000
ABNB,17.748000,17.792000,-0.700000,-0.126000
KEYS,7.296000,7.320000,-2.570000,-0.193000
AETH,6.333000,12.127000,-1.805000,-0.225500
MOON,5.690000,5.700000,-3.390000,-0.200000
AXRP,5.104000,9.699500,-6.000000,-0.619500
POLY,4.596250,8.740500,-5.865000,-0.542250
AXTZ,4.519000,4.584000,-7.630000,-0.378500
ABCH,4.494500,4.479000,-2.130000,-0.097500
